In [1]:
from collections import defaultdict

In [2]:
import jieba

In [3]:
#jieba分词

In [4]:
def cut(string):return list(jieba.cut(string))

In [5]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [6]:
def pat_match(pattern, saying):
    if not pattern or not saying:return[]
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return[]
        else:
            return pat_match(pattern[1:], saying[1:])

In [59]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [8]:
def subsitite(rule, parsed_rules):
    if not rule: return []    
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules) 

In [9]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [10]:
dic = pat_to_dict(got_patterns)

In [11]:
rule = "What if you mean if you got a ?X".split()
#subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

In [77]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [78]:
is_pattern_segment('?*P')

True

In [79]:
from collections import defaultdict

In [80]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [81]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:    #没有输入参数时
        return True
    if not all(a.isalpha() for a in rest[0]):   #rest第一项不全是字母
        return True
    #print(rest)
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])  #匹配余下内容

In [82]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [83]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [123]:
response_pair = { "?*X hello ?*Y": ["Hi, how do you do?"],
     "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    'I need ?X': [ "Why do you neeed ?X" ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [97]:
match = pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

In [98]:
print(match)

[('?X', ['an', 'iPhone'])]


In [99]:
print('------------')

------------


In [100]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(),"I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [101]:
' '.join(subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split()))))

'Why do you neeed an iPhone'

In [102]:
' '.join(subsitite("Hi, how do you do? ?X hello ?Y".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello Bruce".split()))))

'Hi, how do you do? I am mike, hello Bruce'

In [103]:
s = pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello Bruce".split()))

In [104]:
#print(s)

In [124]:
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    print("A:" + saying)
    for key in rules.keys():               
#dict.keys()此方法返回字典中所有可用键的列表;dict = {'Name': 'Maxsu', 'Age': 27}
#  print ("Value : %s" %  dict.keys())当运行上面的程序，它产生以下结果 Value : dict_keys(['Age', 'Name'])
        match_res = pat_match_with_seg(key.split(),saying.split())
        #print(match_res)
        if match_res == fail:
            continue
        
        match_dic = pat_to_dict(match_res)
        #print(match_dic)
        keyReplace = ''.join(subsitite(key.split(),match_dic))
        if keyReplace.replace(' ','') != saying.replace(' ',''):   
            continue
        temDic = {}
        for k,v in match_dic.items():
            if is_pattern_segment(k) :k = k.replace('?*','?')
            temDic[k] = v
        
        result =  " ".join(subsitite(' '.join(rules[key]).split(),temDic))
        print("B:"+ result + '\n')
        return result
    
    
       #Python replace() 方法把字符串中的 old（旧字符串） 替换成 new(新字符串)，如果指定第三个参数max，则替换不超过 max 次
#str.replace(old, new[, max])
# str = "this is string example....wow!!! this is really string";
#如：print str.replace("is", "was");
#   print str.replace("is", "was", 3);
# Output:   thwas was string example....wow!!! thwas was really string
#           thwas was string example....wow!!! thwas is really string

In [125]:
get_response('I need money',response_pair)
get_response('I dont like my dog',response_pair)
get_response('I was late',response_pair)
get_response('I am hellen',response_pair)
get_response('I want to go shopping again',response_pair)   ###结果有误？？？-------???????

A:I need money
B:Why do you neeed money

A:I dont like my dog
B:What bad things did dog do for you?

A:I was late
A:I am hellen
A:I want to go shopping again


In [126]:
arr = cut('?*x我想?*y')
#print(len(arr))

def splitChinese(saying):
    sayingList = cut(saying)
    temArr = []
    #print(sayingList)
    i = 0
    for v in sayingList:
        if sayingList[i] == '?' and (i+ 2) < len(sayingList) and sayingList[i+ 1] == '*' and sayingList[i + 2].isalpha():
            temArr.append('?*' + sayingList[i + 2])
            i = i + 2
        else: 
             if sayingList[i] == '?' and (i+ 1) < len(sayingList) and sayingList[i + 1].isalpha():
                temArr.append('?' + sayingList[i + 1])
                i = i + 1
             else:
                temArr.append(sayingList[i])
        i = i + 1
        if i >= len(sayingList):
            break
    return temArr
arrB = cut("刘敏我想赵晗");
rules = {
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?']
    

}
match = pat_match_with_seg(splitChinese('?*x我想?*y'), arrB)  
#print(match)
match ="".join(subsitite(splitChinese('?*x我想?*y'), pat_to_dict(match)))
#print(match)

def get_response_zh_cn(saying ,pattern):
    print("A:" + saying)
    for key in pattern.keys():
        keyArr = splitChinese(key)
        match_res = pat_match_with_seg(keyArr,cut(saying))
        #print(match_res)
        if match_res == fail:
            continue
        match_dic =  pat_to_dict(match_res)
        #print(match_dic)
        keyReplace = ''.join(subsitite(keyArr,match_dic))
        if keyReplace.replace(' ','') != saying.replace(' ',''):
            continue
        temDic = {}
        for k,v in match_dic.items():
            if is_pattern_segment(k) :k = k.replace('?*','?')
            temDic[k] = v
        #print(temDic)
        result =  "".join(subsitite(splitChinese(''.join(pattern[key])),temDic))
        print("B:"+ result + '\n')
        return result

get_response_zh_cn("刘敏我想减肥",rules)

get_response_zh_cn("亲爱的你好呀",rules)
get_response_zh_cn("刘敏讨厌回家",rules)
get_response_zh_cn("我对不起父母",rules)
get_response_zh_cn("我喜欢他",rules)

#print(splitChinese(''.join(['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'])))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Lee\AppData\Local\Temp\jieba.cache
Loading model cost 1.040 seconds.
Prefix dict has been built succesfully.


A:刘敏我想减肥
A:亲爱的你好呀
A:刘敏讨厌回家
A:我对不起父母
A:我喜欢他
